In [ ]:
import numpy as np

from reachy_sdk import ReachySDK

from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode

In [ ]:
reachy = ReachySDK('192.168.1.21')

Check if all the joints seem to be detected.

In [ ]:
reachy.left_camera.start_autofocus()

## Recording a movement on the right arm

In [ ]:
reachy.turn_off('reachy')
recorded_joints = [
    reachy.r_arm.r_shoulder_pitch,
    reachy.r_arm.r_shoulder_roll,
    reachy.r_arm.r_arm_yaw,
    reachy.r_arm.r_elbow_pitch,
    reachy.r_arm.r_forearm_yaw,
    reachy.r_arm.r_wrist_pitch,
    reachy.r_arm.r_wrist_roll,
]
sampling_frequency = 100  # in Hz
record_duration = 5  # in sec.

import time

trajectories = []

start = time.time()
while (time.time() - start) < record_duration:
    # We here get the present position for all of recorded joints
    current_point = [joint.present_position for joint in recorded_joints]
    # Add this point to the already recorded trajectories
    trajectories.append(current_point)

    time.sleep(1 / sampling_frequency)
print("Recorded!")

In [ ]:
import numpy as np

traj_array = np.array(trajectories)

from reachy_sdk.trajectory import goto

# Set all used joint stiff
for joint in recorded_joints:
    joint.compliant = False

# Create a dict associating a joint to its first recorded position
first_point = dict(zip(recorded_joints, trajectories[0]))

# Goes to the start of the trajectory in 3s
goto(first_point, duration=3.0)
print("First point reached!")

import time

for joints_positions in trajectories:
    for joint, pos in zip(recorded_joints, joints_positions):
        joint.goal_position = pos

    time.sleep(1 / sampling_frequency)
print("Done!")

In [ ]:
reachy.turn_off_smoothly('reachy')